Pipeline:

Data Loading and Preprocessing:


Loads anatomical atlas data (AAL - Automated Anatomical Labeling)
Loads custom ROI (Region of Interest) mappings
Loads two cohorts of iEEG data (MNI_atlas and HUP_atlas)
Applies parameters for patient selection (Engel score threshold and spike threshold)
Feature Extraction

Univariate Features:

Spectral power in different frequency bands (using Welch's method):
Delta (1-4 Hz)
Theta (4-8 Hz)
Alpha (8-13 Hz)
Beta (13-30 Hz)
Gamma (30-80 Hz)
Broadband (1-80 Hz)
Shannon entropy of the signal

Multivariate Features:

Network connectivity measures
Edge-based features
Node-based 

Statistical Analysis:
Z-score normalization of features
Comparison between cohorts
Percentile-based thresholding

In [3]:
import scipy.io as sc
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, butter, lfilter
from scipy.signal.windows import hamming
import nibabel as nib

# from scipy.signal import welch, hamming
from scipy.stats import zscore
from scipy.stats import ttest_ind
from pathlib import Path
import h5py

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import minmax_scale

In [7]:
base_path = '/Users/tereza/nishant/atlas/epi_iEEG_atlas'

Concise

In [10]:
def load_mat_file(file_path):
    """
    Load .mat file with proper error handling and structure inspection
    """
    try:
        # First try loading with scipy.io.loadmat
        data = sc.loadmat(file_path)
        return data
    except NotImplementedError:
        # If that fails, try loading as HDF5
        try:
            with h5py.File(file_path, 'r') as f:
                # Convert to dictionary
                data = {}
                for k, v in f.items():
                    data[k] = np.array(v)
                return data
        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
            return None

def load_and_process_data(base_path):
    """
    Load and process all required data files
    """
    # Define paths
    MNI_atlas_path = os.path.join(base_path, 'Data', 'MNI_atlas.mat')
    HUP_atlas_path = os.path.join(base_path, 'Data', 'HUP_atlas.mat')
    metadata_path = os.path.join(base_path, 'Data', 'metaData.mat')
    custom_atlas_path = os.path.join(base_path, 'Data', 'custom_atlas.xlsx')
    roiAAL_path = os.path.join(base_path, 'Data', 'roiAAL.mat')
    
    # Load data files
    data = {}
    
    # Load MNI atlas
    print("Loading MNI atlas...")
    data['MNI_atlas'] = load_mat_file(MNI_atlas_path)
    
    # Load HUP atlas
    print("Loading HUP atlas...")
    data['HUP_atlas'] = load_mat_file(HUP_atlas_path)
    
    # Load metadata
    print("Loading metadata...")
    data['metaData'] = load_mat_file(metadata_path)
    
    # Load custom atlas
    print("Loading custom atlas...")
    data['customAAL'] = pd.read_excel(custom_atlas_path)
    
    # Load and process roiAAL
    print("Loading roiAAL...")
    roiAAL = load_mat_file(roiAAL_path)
    
    # Print available keys in roiAAL
    print("\nKeys in roiAAL:", roiAAL.keys())
    
    # Try to extract roiAAL data
    roiAAL_data = None
    if 'roiAAL' in roiAAL:
        roiAAL_data = roiAAL['roiAAL']
    else:
        # Inspect other keys
        for key in roiAAL:
            if not key.startswith('__'):
                print(f"\nInspecting key: {key}")
                print("Data type and shape:", type(roiAAL[key]), np.shape(roiAAL[key]))
                roiAAL_data = roiAAL[key]
    
    # If we found the data, create DataFrame
    if roiAAL_data is not None:
        try:
            data['roiAAL_df'] = pd.DataFrame(roiAAL_data, 
                                           columns=['Sno', 'parcel', 'Regions', 'Lobes', 'isSideLeft'])
            print("\nSuccessfully created roiAAL DataFrame with shape:", data['roiAAL_df'].shape)
        except Exception as e:
            print(f"\nError creating DataFrame: {str(e)}")
            print("roiAAL_data shape:", np.shape(roiAAL_data))
            print("roiAAL_data type:", type(roiAAL_data))
            # If it's a structured array, print field names
            if hasattr(roiAAL_data, 'dtype') and roiAAL_data.dtype.names is not None:
                print("Available fields:", roiAAL_data.dtype.names)
    
    return data

# Test the loading function
if __name__ == "__main__":
    base_path = '/Users/tereza/nishant/atlas/epi_iEEG_atlas'
    data = load_and_process_data(base_path)
    
    # Print summary of loaded data
    print("\nSummary of loaded data:")
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            print(f"{key}: DataFrame with shape {value.shape}")
        elif isinstance(value, dict):
            print(f"{key}: Dictionary with keys {list(value.keys())}")
        else:
            print(f"{key}: {type(value)}")

Loading MNI atlas...
Loading HUP atlas...
Loading metadata...
Loading custom atlas...
Loading roiAAL...

Keys in roiAAL: dict_keys(['__header__', '__version__', '__globals__', 'None', '__function_workspace__'])

Inspecting key: None
Data type and shape: <class 'scipy.io.matlab._mio5_params.MatlabOpaque'> (1,)

Successfully created roiAAL DataFrame with shape: (1, 5)

Summary of loaded data:
MNI_atlas: Dictionary with keys ['__header__', '__version__', '__globals__', 'AgeAtTimeOfStudy', 'ChannelName', 'ChannelPosition', 'ChannelRegion', 'ChannelType', 'Data_N2', 'Data_N3', 'Data_R', 'Data_W', 'FacesLeft', 'FacesRight', 'Gender', 'Hemisphere', 'NodesLeft', 'NodesLeftInflated', 'NodesRegionLeft', 'NodesRegionRight', 'NodesRight', 'NodesRightInflated', 'Patient', 'RegionName', 'SamplingFrequency']
HUP_atlas: Dictionary with keys ['__header__', '__version__', '__globals__', 'SamplingFrequency', 'depth_elecs', 'mni_coords', 'patient_no', 'resected_ch', 'soz_ch', 'spike_24h', 'wake_clip']
met

Translating MATLAB->Python

mergeROIs:
- Merges multiple ROIs (Regions of Interest) into custom atlases
- Handles parcel assignments and coordinate transformations
- Creates a unified atlas for subsequent analysis

implant2roi:
- Maps electrode coordinates to specific brain regions/ROIs
- Performs coordinate system transformations (CRS to RAS)
- Creates electrode-to-ROI mapping tables

get_norm_psd:
- Computes normalized power spectral density for iEEG data
- Extracts frequency band powers (delta, theta, alpha, beta, gamma)
- Applies noise filtering and normalization

get_norm_entropy:
- Calculates Shannon entropy of the iEEG signals
- Applies bandpass and notch filtering
- Adds entropy features to the analysis

plot_ieeg_atlas:
- Creates visualizations of iEEG data mapped onto brain atlases
- Computes regional statistics (mean, std) for different frequency bands
- Handles ROI-based data aggregation

compare_ieeg_atlas:
- Compares metrics between MNI and HUP atlases
- Generates comparison plots (electrode counts, bandpower distributions)
- Combines and normalizes data from both atlases

make_seizure_free/make_seizure_free_abr:
- Filters patients based on surgical outcomes (Engel scores)
- Identifies healthy vs abnormal tissue regions
- Creates subsets for analysis based on clinical criteria

In [4]:
#all custom functions 

# Python translation of MATLAB's `mergeROIs` function
def merge_rois(customAAL, roiAAL, atlas):
    
    # Convert inputs into appropriate types (assuming customAAL and roiAAL are Pandas DataFrames)
    for roi in range(len(customAAL)):
        # Assign parcel1 from roiAAL based on customAAL.Roi1
        customAAL.loc[roi, 'parcel1'] = roiAAL.loc[customAAL.loc[roi, 'Roi1'], 'parcel']
        
        # Handle Roi2 assignment
        if np.isnan(customAAL.loc[roi, 'Roi2']):
            customAAL.loc[roi, 'parcel2'] = np.nan
        else:
            customAAL.loc[roi, 'parcel2'] = roiAAL.loc[customAAL.loc[roi, 'Roi2'], 'parcel']
            atlas['data'][atlas['data'] == customAAL.loc[roi, 'parcel2']] = customAAL.loc[roi, 'parcel1']
        
        # Handle Roi3 assignment
        if np.isnan(customAAL.loc[roi, 'Roi3']):
            customAAL.loc[roi, 'parcel3'] = np.nan
        else:
            customAAL.loc[roi, 'parcel3'] = roiAAL.loc[customAAL.loc[roi, 'Roi3'], 'parcel']
            atlas['data'][atlas['data'] == customAAL.loc[roi, 'parcel3']] = customAAL.loc[roi, 'parcel1']
    
    # Handle inclusion/exclusion logic
    included = np.concatenate((customAAL['Roi1'], customAAL['Roi2'], customAAL['Roi3']))
    included = included[~np.isnan(included)]  # Remove NaN values
    
    excluded = np.setxor1d(roiAAL['Sno'], included)
    atlas['data'][np.isin(atlas['data'], roiAAL['parcel'][excluded])] = 0

    # Create atlasCustom and roiAALcustom as outputs
    atlasCustom = atlas
    
    roiAALcustom = {}
    roiAALcustom['Sno'] = np.arange(1, len(customAAL) + 1)
    roiAALcustom['Regions'] = customAAL['Roi_name']
    roiAALcustom['Lobes'] = customAAL['Lobes']
    roiAALcustom['isSideLeft'] = customAAL['Roi_name'].str.endswith('_L')
    roiAALcustom['parcel'] = customAAL['parcel1']
    
    # Calculate coordinates (CRS to RAS transformation)
    xyz = []
    for roi in range(len(customAAL)):
        indices = np.argwhere(atlas['data'] == roiAALcustom['parcel'][roi])
        CRS = np.hstack([indices, np.full((indices.shape[0], 1), roiAALcustom['parcel'][roi])])
        
        RAS = np.dot(atlas['hdr']['Transform']['T'].T, np.hstack([CRS[:, :3], np.ones((CRS.shape[0], 1))]).T).T
        RAS = RAS[:, :3]
        xyz.append(RAS.mean(axis=0))
    xyz = np.array(xyz)
    
    roiAALcustom['x'] = xyz[:, 0]
    roiAALcustom['y'] = xyz[:, 1]
    roiAALcustom['z'] = xyz[:, 2]

    # Convert roiAALcustom to Pandas DataFrame for easier use
    roiAALcustom = pd.DataFrame(roiAALcustom)
    
    return atlasCustom, roiAALcustom

# Python translation of MATLAB's `implant2ROI` function
def implant2roi(atlas, electrodeCord, patientNum):

    # Get unique ROI from atlas excluding zeros
    nROI = np.unique(atlas['data'][atlas['data'] != 0])
    
    # Get voxel coordinates in CRS format
    CRScord = []
    for roi in nROI:
        indices = np.argwhere(atlas['data'] == roi)
        CRS = np.hstack([indices, np.full((indices.shape[0], 1), roi)])
        CRScord.append(CRS)
    CRScord = np.vstack(CRScord)

    # Convert CRS to RAS using atlas transform
    RAScord = np.dot(atlas['hdr']['Transform']['T'].T, np.hstack([CRScord[:, :3], np.ones((CRScord.shape[0], 1))]).T).T
    RAScord = RAScord[:, :3]
    RAScord = np.hstack([RAScord, CRScord[:, 3:]])

    # Identify nearest neighbor of each electrode
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(RAScord[:, :3])
    distances, indices = nbrs.kneighbors(electrodeCord)
    atlasROI = RAScord[indices.flatten(), 3]

    # Map to ROI numbers
    roiNum = [np.where(atlas['tbl']['parcel'] == roi)[0][0] for roi in atlasROI]
    
    # Create electrode2roi table
    electrode2roi = pd.DataFrame({'roiNum': roiNum, 'atlasROI': atlasROI, 'patientNum': patientNum})
    
    return electrode2roi

# Python translation of MATLAB's `getNormPSD` function
def get_norm_psd(iEEGnormal, data_timeS, SamplingFrequency):

    # Get sampling frequency, time domain data, window length, and NFFT
    Fs = SamplingFrequency
    data_seg = data_timeS[:Fs*60, :]
    window = Fs * 2
    NFFT = window

    # Compute PSD
    f, psd = welch(data_seg, fs=Fs, window=hamming(window), nperseg=window, noverlap=0, nfft=NFFT, axis=0)

    # Filter out noise frequency between 57.7Hz and 62.5Hz
    idx = (f >= 57.5) & (f <= 62.5)
    psd[idx, :] = 0
    f = f[~idx]

    # Compute bandpower
    delta = np.trapz(psd[(f >= 1) & (f < 4)], f[(f >= 1) & (f < 4)], axis=0)
    theta = np.trapz(psd[(f >= 4) & (f < 8)], f[(f >= 4) & (f < 8)], axis=0)
    alpha = np.trapz(psd[(f >= 8) & (f < 13)], f[(f >= 8) & (f < 13)], axis=0)
    beta = np.trapz(psd[(f >= 13) & (f < 30)], f[(f >= 13) & (f < 30)], axis=0)
    gamma = np.trapz(psd[(f >= 30) & (f < 80)], f[(f >= 30) & (f < 80)], axis=0)
    broad = np.trapz(psd[(f >= 1) & (f < 80)], f[(f >= 1) & (f < 80)], axis=0)

    # Log transform
    deltalog = np.log10(delta + 1)
    thetalog = np.log10(theta + 1)
    alphalog = np.log10(alpha + 1)
    betalog = np.log10(beta + 1)
    gammalog = np.log10(gamma + 1)
    broadlog = np.log10(broad + 1)

    # Total power
    tPow = deltalog + thetalog + alphalog + betalog + gammalog

    # Relative power
    deltaRel = deltalog / tPow
    thetaRel = thetalog / tPow
    alphaRel = alphalog / tPow
    betaRel = betalog / tPow
    gammaRel = gammalog / tPow

    # Append to iEEGnormal
    iEEGnormal = pd.concat([iEEGnormal, pd.DataFrame({'delta': deltaRel, 'theta': thetaRel, 'alpha': alphaRel, 'beta': betaRel, 'gamma': gammaRel, 'broad': broadlog})], axis=1)

    return iEEGnormal

# Python translation of MATLAB's `getNormEntropy` function
def get_norm_entropy(iEEGnormal, data_timeS, SamplingFrequency):

    # Helper functions for filtering
    def eeg_filter(data, cutoff, fs, btype, order=3):
        nyquist = 0.5 * fs
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype=btype, analog=False)
        return lfilter(b, a, data, axis=0)

    # Get sampling frequency, time domain data, window length, and NFFT
    Fs = SamplingFrequency
    data_seg = data_timeS[:Fs*60, :]

    # Apply filters
    data_segbb = eeg_filter(data_seg, 80, Fs, 'low')
    data_segbb = eeg_filter(data_segbb, 1, Fs, 'high')
    data_segbb_notch = eeg_filter(data_segbb, 60, Fs, 'bandstop')

    # Compute Shannon entropy
    entropy = np.array([np.log10(-np.sum(p * np.log2(p)) + 1) for p in data_segbb_notch.T])

    # Append entropy to iEEGnormal
    iEEGnormal = pd.concat([iEEGnormal, pd.DataFrame({'entropy': entropy})], axis=1)

    return iEEGnormal


# Python translation of MATLAB's `plotiEEGatlas` function
def plot_ieeg_atlas(iEEGnormal, atlas, plot_option='noplot'):
    
    # Create normAtlas dictionary to store data
    normAtlas = {}
    normAtlas['roi'] = atlas['tbl']['Sno']
    normAtlas['name'] = atlas['tbl']['Regions']
    normAtlas['lobe'] = atlas['tbl']['Lobes']
    normAtlas['isSideLeft'] = atlas['tbl']['isSideLeft']

    # Calculate metrics for each ROI
    nROIs = len(atlas['tbl']['Sno'])
    for roi in range(nROIs):
        idx = iEEGnormal['roiNum'] == roi
        
        # Number of electrodes in each region
        normAtlas.setdefault('nElecs', []).append(np.sum(idx))
        
        # Mean and standard deviation for delta, theta, alpha, beta, gamma, and broad bands
        for band in ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broad']:
            normAtlas.setdefault(band, []).append(iEEGnormal[band][idx].tolist())
            normAtlas.setdefault(f'{band}Mean', []).append(np.mean(iEEGnormal[band][idx]))
            normAtlas.setdefault(f'{band}Std', []).append(np.std(iEEGnormal[band][idx]))
    
    # Convert to DataFrame for easier processing
    normAtlas = pd.DataFrame(normAtlas)
    
    # Remove regions with no electrodes
    normAtlas = normAtlas[normAtlas['nElecs'] > 0]

    # Plotting logic
    if plot_option == 'plot':
                
        bands = ['deltaMean', 'thetaMean', 'alphaMean', 'betaMean', 'gammaMean', 'broadMean']
        for i, band in enumerate(bands):
            normAtlas['scaled'] = minmax_scale(normAtlas[band])
            nodeVal = pd.DataFrame({
                'x': normAtlas['x'],
                'y': normAtlas['y'],
                'z': normAtlas['z'],
                'value': normAtlas['scaled'],
                'color': normAtlas['scaled']
            })
            nodeVal.to_csv('nodeVal.node', sep='\t', index=False, header=False)
            
            # Brain network visualization (placeholder for actual implementation)
            if i < 5:
                # Call some equivalent of BrainNet_MapCfg function for plotting
                pass  # Replace with Python visualization library
            else:
                # Call some equivalent of BrainNet_MapCfg function for plotting (high frequency config)
                pass  # Replace with Python visualization library
    
    return normAtlas

# Python translation of MATLAB's `compareiEEGatlas` function
def compare_ieeg_atlas(normMNIAtlas, normHUPAtlas, plot_option='noplot'):
    
    # Filter out matching ROIs
    MNI = normMNIAtlas[normMNIAtlas['roi'].isin(normHUPAtlas['roi'])]
    HUP = normHUPAtlas[normHUPAtlas['roi'].isin(normMNIAtlas['roi'])]
    
    if plot_option == 'plot':
        # Compare the number of electrodes between atlases
        plt.figure()
        plt.barh(np.arange(len(MNI)), [MNI['nElecs'], HUP['nElecs']], label=['MNI', 'HUP'])
        plt.xlabel('Number of electrodes')
        plt.yticks(np.arange(len(MNI)), MNI['name'])
        plt.gca().tick_params(axis='y', which='both', labelsize='small')
        plt.tight_layout()
        plt.savefig('Figure/nElec.pdf', format='pdf', dpi=300)

        # Get the effect size across regions
        plt.figure()
        bands = ['deltaMean', 'thetaMean', 'alphaMean', 'betaMean', 'gammaMean', 'broadMean']
        data = []
        for band in bands:
            data.append(np.vstack([MNI[band].values, HUP[band].values]).T)
        data = np.concatenate(data, axis=1)
        
        plt.boxplot(data[:, :10])  # Simplified scatter representation
        plt.xticks(np.arange(1, 11), [f'{band}MNI' for band in bands[:5]] + [f'{band}HUP' for band in bands[:5]], rotation=45)
        plt.ylabel('Normalized relative bandpower')
        plt.tight_layout()
        plt.savefig('Figure/bandPow.pdf', format='pdf', dpi=300)

    # Combine HUP with MNI for missing ROIs
    newROIhup = normHUPAtlas[~normHUPAtlas['roi'].isin(normMNIAtlas['roi'])]
    normMNIAtlas = pd.concat([normMNIAtlas, newROIhup], ignore_index=True)
    
    # Update metrics for common ROIs
    commonROIhup = normHUPAtlas[normHUPAtlas['roi'].isin(normMNIAtlas['roi'])]
    lbl = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broad']
    for _, row in commonROIhup.iterrows():
        id = normMNIAtlas.index[normMNIAtlas['roi'] == row['roi']][0]
        normMNIAtlas.at[id, 'nElecs'] += row['nElecs']
        for band in lbl:
            combined = normMNIAtlas.at[id, band] + row[band]
            normMNIAtlas.at[id, band] = combined
            normMNIAtlas.at[id, f'{band}Mean'] = np.mean(combined)
            normMNIAtlas.at[id, f'{band}Std'] = np.std(combined)
    
    normMNIAtlas = normMNIAtlas.sort_values(by='roi').reset_index(drop=True)
    
    return normMNIAtlas

def node_abr_edge(abr_conn, ieeg_hup_all, percentile_thres):
    fbands = [col for col in abr_conn.columns if col.endswith('_z')]
    ieeg_abr = []
    
    for s in range(len(abr_conn)):
        node_abr = []
        for f in fbands:
            adj = abr_conn.loc[s, f]
            node_abr.append(np.percentile(adj, percentile_thres, axis=1))
        ieeg_abr.append(np.column_stack(node_abr))
    
    ieeg_abr = np.vstack(ieeg_abr)
    ieeg_hup_all = pd.concat([ieeg_hup_all, pd.DataFrame(ieeg_abr, columns=[f + '_coh' for f in fbands])], axis=1)
    
    return ieeg_hup_all

def edgeslist_to_abr_conn(pat_connection, hup_atlas_all):
    n_sub = pat_connection['patientNum'].unique()
    fbands = [col for col in pat_connection.columns if col.endswith('_z')]
    abr_conn = {'patientNum': []}

    for s in n_sub:
        abr_conn['patientNum'].append(s)
        n_elec = (hup_atlas_all['patient_no'] == s).sum()
        for f in fbands:
            edges = pat_connection.loc[pat_connection['patientNum'] == s, f].values
            adj = np.reshape(edges, (n_elec, n_elec))
            adj[np.isnan(adj)] = 0
            abr_conn.setdefault(f, []).append(np.abs(adj))
    
    return pd.DataFrame(abr_conn)

def univariate_abr(norm_mni_hup_atlas, ieeg_hup_all):
    rel_pow_z = []

    for n_elec in range(len(ieeg_hup_all)):
        roi_num = ieeg_hup_all.loc[n_elec, 'roiNum']
        norm_mu = norm_mni_hup_atlas.loc[roi_num, ['deltaMean', 'thetaMean', 'alphaMean', 'betaMean', 'gammaMean', 'broadMean']].values
        norm_sigma = norm_mni_hup_atlas.loc[roi_num, ['deltaStd', 'thetaStd', 'alphaStd', 'betaStd', 'gammaStd', 'broadStd']].values
        rel_pow = ieeg_hup_all.loc[n_elec, ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broad']].values
        rel_pow_z.append(np.abs((rel_pow - norm_mu) / norm_sigma))

    rel_pow_z = np.array(rel_pow_z)
    ieeg_hup_all_z = pd.concat([ieeg_hup_all.iloc[:, :3], pd.DataFrame(rel_pow_z, columns=['delta_z', 'theta_z', 'alpha_z', 'beta_z', 'gamma_z', 'broad_z'])], axis=1)
    
    return ieeg_hup_all_z

def make_seizure_free_abr(hup_atlas_all, meta_data, engel_sf_thres, spike_thresh):
    outcomes = np.nanmax(meta_data[['Engel_6_mo', 'Engel_12_mo']].values, axis=1)
    sf_patients = np.where(outcomes <= engel_sf_thres)[0]

    sf_patients_ieeg = hup_atlas_all['patient_no'].isin(sf_patients)
    resected_sf_ieeg = sf_patients_ieeg & hup_atlas_all['resected_ch']
    soz_spared_sf_ieeg = resected_sf_ieeg & hup_atlas_all['soz_ch']
    abnormal_ieeg = soz_spared_sf_ieeg & (hup_atlas_all['spike_24h'] > spike_thresh)

    hup_abr_atlas = hup_atlas_all.loc[abnormal_ieeg].copy()
    hup_abr_atlas['SamplingFrequency'] = hup_atlas_all['SamplingFrequency']
    
    return hup_abr_atlas

def make_seizure_free(hup_atlas_all, meta_data, engel_sf_thres, spike_thresh):
    outcomes = np.nanmax(meta_data[['Engel_6_mo', 'Engel_12_mo']].values, axis=1)
    sf_patients = np.where(outcomes <= engel_sf_thres)[0]

    sf_patients_ieeg = hup_atlas_all['patient_no'].isin(sf_patients)
    spared_sf_ieeg = sf_patients_ieeg & ~hup_atlas_all['resected_ch']
    not_soz_spared_sf_ieeg = spared_sf_ieeg & ~hup_atlas_all['soz_ch']
    healthy_ieeg = not_soz_spared_sf_ieeg & (hup_atlas_all['spike_24h'] < spike_thresh)

    hup_atlas = hup_atlas_all.loc[healthy_ieeg].copy()
    hup_atlas['SamplingFrequency'] = hup_atlas_all['SamplingFrequency']
    
    return hup_atlas

# Python translation of MATLAB's `mainUnivar` function
def main_univar(metaData, atlas, MNI_atlas, HUP_atlasAll, EngelSFThres, spikeThresh):
    
    # MNI atlas electrode to ROI
    electrodeCord = MNI_atlas['ChannelPosition']
    patientNum = MNI_atlas['Patient']
    iEEG_mni = implant2roi(atlas, electrodeCord, patientNum)

    # MNI atlas normalised bandpower
    data_MNI = MNI_atlas['Data_W']
    SamplingFrequency = MNI_atlas['SamplingFrequency']
    iEEG_mni = get_norm_psd(iEEG_mni, data_MNI, SamplingFrequency)

    # Seizure-free HUP atlas electrode to ROI
    HUP_atlas = make_seizure_free(HUP_atlasAll, metaData, EngelSFThres, spikeThresh)
    electrodeCord = HUP_atlas['mni_coords']
    patientNum = HUP_atlas['patient_no']
    iEEG_hup = implant2roi(atlas, electrodeCord, patientNum)

    # HUP atlas normalised bandpower
    data_HUP = HUP_atlas['wake_clip']
    SamplingFrequency = HUP_atlas['SamplingFrequency']
    iEEG_hup = get_norm_entropy(iEEG_hup, data_HUP, SamplingFrequency)

    # Visualise MNI and HUP atlas
    norm_mni_atlas = plot_ieeg_atlas(iEEG_mni, atlas, plot_option='noplot')
    norm_hup_atlas = plot_ieeg_atlas(iEEG_hup, atlas, plot_option='noplot')
    norm_MNI_HUP_Atlas = compare_ieeg_atlas(norm_mni_atlas, norm_hup_atlas, plot_option='plot')

    # Seizure-free HUP atlas electrode to ROI for all patients
    electrodeCord = HUP_atlasAll['mni_coords']
    patientNum = HUP_atlasAll['patient_no']
    iEEG_hup_all = implant2roi(atlas, electrodeCord, patientNum)
    data_HUP_all = HUP_atlasAll['wake_clip']
    SamplingFrequency = HUP_atlasAll['SamplingFrequency']
    iEEG_hup_all = get_norm_psd(iEEG_hup_all, data_HUP_all, SamplingFrequency)

    # Abnormal HUP atlas
    HUP_abr_atlas = make_seizure_free_abr(HUP_atlasAll, metaData, EngelSFThres, spikeThresh)
    electrodeCord = HUP_abr_atlas['mni_coords']
    patientNum = HUP_abr_atlas['patient_no']
    iEEG_hup_abr = implant2roi(atlas, electrodeCord, patientNum)
    data_HUP_abr = HUP_abr_atlas['wake_clip']
    SamplingFrequency = HUP_abr_atlas['SamplingFrequency']
    iEEG_hup_abr = get_norm_psd(iEEG_hup_abr, data_HUP_abr, SamplingFrequency)
    abrnorm_hup_atlas = plot_ieeg_atlas(iEEG_hup_abr, atlas, plot_option='noplot')
    abrnorm_hup_atlas = abrnorm_hup_atlas.sort_values(by='nElecs', ascending=False)

    # Address reviewer 1 comment
    # pGrp, d = rev1_actual_pow(HUP_abr_atlas, iEEG_hup_abr, HUP_atlas, MNI_atlas, iEEG_hup, iEEG_mni)
    # d = rev1_surg_outcome(HUP_atlasAll, iEEG_hup_all, metaData)

    return norm_MNI_HUP_Atlas, iEEG_hup_all, abrnorm_hup_atlas

Main

In [8]:
def main():
    # Parameters
    EngelSFThres = 1.1
    spikeThresh = 24

    # Load and prepare atlas data
    print("Loading atlas data...")
    atlas = {}
    atlas_path = os.path.join(base_path, 'Data', 'AAL.nii.gz')
    atlas['data'] = nib.load(atlas_path).get_fdata()
    atlas['data'][atlas['data'] > 9000] = 0
    atlas['hdr'] = nib.load(atlas_path).header
    
    # Load other required data
    print("Loading other data files...")
    MNI_atlas = sc.loadmat(os.path.join(base_path, 'Data', 'MNI_atlas.mat'))
    HUP_atlas = sc.loadmat(os.path.join(base_path, 'Data', 'HUP_atlas.mat'))
    metaData = sc.loadmat(os.path.join(base_path, 'Data', 'metaData.mat'))
    customAAL = pd.read_excel(os.path.join(base_path, 'Data', 'custom_atlas.xlsx'))
    roiAAL = sc.loadmat(os.path.join(base_path, 'Data', 'roiAAL.mat'))

    # Create custom atlas
    print("Creating custom atlas...")
    atlasCustom, roiAALcustom = merge_rois(customAAL, roiAAL, atlas)
    atlas['tbl'] = roiAAL
    atlasCustom['tbl'] = roiAALcustom
    atlas = atlasCustom

    # Run univariate analysis
    print("Running univariate analysis...")
    norm_MNI_HUP_Atlas, iEEG_hup_all, abrnorm_hup_atlas = main_univar(
        metaData, atlas, MNI_atlas, HUP_atlas, EngelSFThres, spikeThresh
    )

    # Generate univariate z-scores
    print("Computing univariate z-scores...")
    iEEG_hup_all_z = univariate_abr(norm_MNI_HUP_Atlas, iEEG_hup_all)

    # Create output directory if it doesn't exist
    os.makedirs(os.path.join(base_path, 'Figures'), exist_ok=True)

    # Generate and save comparison plots
    print("Generating comparison plots...")
    plot_comparisons(norm_MNI_HUP_Atlas, iEEG_hup_all_z, base_path)

    return norm_MNI_HUP_Atlas, iEEG_hup_all_z, abrnorm_hup_atlas

def plot_comparisons(norm_MNI_HUP_Atlas, iEEG_hup_all_z, base_path):
    """Generate comprehensive comparison plots"""
    
    # Plot 1: Feature distributions across cohorts
    plt.figure(figsize=(12, 6))
    bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    plt.boxplot([iEEG_hup_all_z[f'{band}_z'] for band in bands])
    plt.xticks(range(1, len(bands) + 1), bands)
    plt.ylabel('Z-scored Power')
    plt.title('Distribution of Frequency Band Powers')
    plt.savefig(os.path.join(base_path, 'Figures', 'band_powers.pdf'))
    plt.close()

    # Plot 2: ROI comparison
    plt.figure(figsize=(15, 8))
    roi_means = norm_MNI_HUP_Atlas.groupby('roi')[bands].mean()
    roi_means.plot(kind='bar')
    plt.title('Mean Band Power by ROI')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(base_path, 'Figures', 'roi_comparison.pdf'))
    plt.close()

# If running in a notebook, you can run this directly:
try:
    norm_MNI_HUP_Atlas, iEEG_hup_all_z, abrnorm_hup_atlas = main()
    print("Analysis completed successfully.")
    print(f"Results saved in {os.path.join(base_path, 'Figures')}")
except Exception as e:
    print(f"Error during analysis: {str(e)}")

Loading atlas data...
Loading other data files...
Creating custom atlas...
Error during analysis: 'dict' object has no attribute 'loc'


In [15]:
# new 

def main():
    # Parameters
    EngelSFThres = 1.1
    spikeThresh = 24
    
    # Load data using the load_and_process_data function
    base_path = '/Users/tereza/nishant/atlas/epi_iEEG_atlas'
    loaded_data = load_and_process_data(base_path)
    
    # Extract the loaded data
    MNI_atlas = loaded_data['MNI_atlas']
    HUP_atlas = loaded_data['HUP_atlas']
    metaData = loaded_data['metaData']
    customAAL = loaded_data['customAAL']
    roiAAL = loaded_data['roiAAL_df']  # Note: using roiAAL_df instead of raw roiAAL

    # Load and prepare atlas data
    print("Loading atlas data...")
    atlas = {}
    atlas_path = os.path.join(base_path, 'Data', 'AAL.nii.gz')
    atlas['data'] = nib.load(atlas_path).get_fdata()
    atlas['data'][atlas['data'] > 9000] = 0
    atlas['hdr'] = nib.load(atlas_path).header

    # Create custom atlas
    print("Creating custom atlas...")
    atlasCustom, roiAALcustom = merge_rois(customAAL, roiAAL, atlas)
    atlas['tbl'] = roiAAL
    atlasCustom['tbl'] = roiAALcustom
    atlas = atlasCustom

    # Run univariate analysis
    print("Running univariate analysis...")
    norm_MNI_HUP_Atlas, iEEG_hup_all, abrnorm_hup_atlas = main_univar(
        metaData, atlas, MNI_atlas, HUP_atlas, EngelSFThres, spikeThresh
    )

    # Generate univariate z-scores
    print("Computing univariate z-scores...")
    iEEG_hup_all_z = univariate_abr(norm_MNI_HUP_Atlas, iEEG_hup_all)

    # Create output directory if it doesn't exist
    os.makedirs(os.path.join(base_path, 'figures'), exist_ok=True)  # Note: changed Figures to figures

    # Generate and save comparison plots
    print("Generating comparison plots...")
    plot_comparisons(norm_MNI_HUP_Atlas, iEEG_hup_all_z, base_path)

    return norm_MNI_HUP_Atlas, iEEG_hup_all_z, abrnorm_hup_atlas

norm_MNI_HUP_Atlas, iEEG_hup_all_z, abrnorm_hup_atlas = main()

Loading MNI atlas...
Loading HUP atlas...
Loading metadata...
Loading custom atlas...
Loading roiAAL...

Keys in roiAAL: dict_keys(['__header__', '__version__', '__globals__', 'None', '__function_workspace__'])

Inspecting key: None
Data type and shape: <class 'scipy.io.matlab._mio5_params.MatlabOpaque'> (1,)

Successfully created roiAAL DataFrame with shape: (1, 5)
Loading atlas data...
Creating custom atlas...


KeyError: 1